<a href="https://colab.research.google.com/github/UEPP40/PUM/blob/wilmanski/Zaj%C4%99cia2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd

In [11]:
df = pd.read_csv('train.csv')

<ipython-input-11-cd75a77447fc>:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('train.csv')


Wymiary

In [13]:
print(df.shape)

(100000, 28)


Wyrzucenie duplikatów

In [9]:
print(df.drop_duplicates())

            ID Customer_ID     Month           Name   Age          SSN  \
0       0x1602   CUS_0xd40   January  Aaron Maashoh    23  821-00-0265   
1       0x1603   CUS_0xd40  February  Aaron Maashoh    23  821-00-0265   
2       0x1604   CUS_0xd40     March  Aaron Maashoh  -500  821-00-0265   
3       0x1605   CUS_0xd40     April  Aaron Maashoh    23  821-00-0265   
4       0x1606   CUS_0xd40       May  Aaron Maashoh    23  821-00-0265   
...        ...         ...       ...            ...   ...          ...   
53925  0x151f7  CUS_0x4460      June         Daveyx    54  909-64-1351   
53926  0x151f8  CUS_0x4460      July         Daveyx    54  909-64-1351   
53927  0x151f9  CUS_0x4460    August         Daveyx    54  909-64-1351   
53928  0x151fe  CUS_0xc0b3   January            NaN    44  973-12-0733   
53929  0x151ff  CUS_0xc0b3  February         Grantg    44  973-12-0733   

       Occupation Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  \
0       Scientist      19114.12   

In [14]:
df = df.astype({col: 'string' for col in df.select_dtypes(['object']).columns})
print(df.dtypes)

ID                           string
Customer_ID                  string
Month                        string
Name                         string
Age                          string
SSN                          string
Occupation                   string
Annual_Income                string
Monthly_Inhand_Salary       float64
Num_Bank_Accounts             int64
Num_Credit_Card               int64
Interest_Rate                 int64
Num_of_Loan                  string
Type_of_Loan                 string
Delay_from_due_date           int64
Num_of_Delayed_Payment       string
Changed_Credit_Limit         string
Num_Credit_Inquiries        float64
Credit_Mix                   string
Outstanding_Debt             string
Credit_Utilization_Ratio    float64
Credit_History_Age           string
Payment_of_Min_Amount        string
Total_EMI_per_month         float64
Amount_invested_monthly      string
Payment_Behaviour            string
Monthly_Balance              string
Credit_Score                